In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Common import spawn_camera, spawn_vehicle, preprocess_image, spawn_camera_depth, riconosci_parcheggio1

pygame 2.6.1 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()

In [6]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(-1, -25, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=90, pitch=-40)))
run = True

In [7]:
def camera_callback(image):
    global run
    prediction=riconosci_parcheggio1(preprocess_image(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))))
    print(prediction)
    if prediction > 0.65:
        run = False
camera_rigth.listen(lambda image: camera_callback(image))

0.318209707736969
0.3119812607765198
0.3117683231830597
0.313111811876297
0.3123202919960022
0.3143412470817566
0.3132256269454956
0.3119325339794159
0.31212562322616577
0.3120584189891815
0.31161952018737793
0.3105846345424652
0.31124767661094666
0.31048035621643066
0.31149476766586304
0.31223049759864807
0.3105847239494324
0.31088462471961975
0.3104802966117859
0.3110557198524475
0.31223058700561523
0.3112787902355194
0.31258711218833923
0.31150296330451965


In [8]:
control = carla.VehicleControl()
control.steer = 0.0 
control.throttle = 0.5 
control.brake = 0.0  

print(vehicle.get_velocity())

while run:
        vehicle.apply_control(control)
        time.sleep(0.1)
        print(vehicle.get_velocity().length())
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
time.sleep(1.1)

control.throttle = 0.5 
control.brake = 0.0 
i = 0 
while i<25:
        vehicle.apply_control(control)
        i=i+1
        time.sleep(0.1)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
time.sleep(0.1)

i = 0 
while i<80:
        control = carla.VehicleControl(throttle=0.3, steer=0.8, brake=0.0, reverse=True)
        vehicle.apply_control(control)
        i=i+1
        time.sleep(0.1)
control = carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0, reverse=False)
vehicle.apply_control(control)
time.sleep(4.1)
camera_retro.destroy()
camera_rigth.destroy()
vehicle.destroy()

0.3099883198738098
Vector3D(x=0.000000, y=0.000000, z=0.000000)
0.3114173710346222
0.0033773821778595448
0.3119039535522461
0.0019156273920089006
0.31087926030158997
0.0011268837843090296
0.3117709159851074
0.00021439213014673442
0.00030566888744942844
0.31110572814941406
0.0001787807559594512
0.31241992115974426
0.0001838470489019528
0.31076377630233765
0.00027491175569593906
0.3108593225479126
0.0005061926203779876
0.31185775995254517
0.8515187501907349
0.310165673494339
1.3773572444915771
0.31658077239990234
1.5613152980804443
1.6637064218521118
0.32564303278923035
1.7462555170059204
0.33394068479537964
1.8033753633499146
0.3586132228374481
1.8869941234588623
0.3939630389213562
1.9600365161895752
0.3746117353439331
2.0214319229125977
0.32841387391090393
2.077064037322998
0.2974386513233185
2.1291801929473877
2.179697036743164
0.2929826080799103
2.226400136947632
0.29533785581588745
2.270232677459717
0.2889092266559601
2.306925058364868
0.33983251452445984
2.349151134490967
2.3808004

True

0.2847500145435333


: 

In [20]:
import time

# Define the target speed in m/s
target_speed_mps = 10 / 3.6  # 15 km/h to m/s

control = carla.VehicleControl()
control.steer = 0.0
control.brake = 0.0

target_distance = 5.0
distance_travelled = 0.0

while distance_travelled < target_distance:
    # Get the current speed of the vehicle
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()  # Speed in m/s
    distance_travelled += current_speed_mps * 0.1
    # Compute the speed error (difference between current and target speed)
    speed_error = target_speed_mps - current_speed_mps
    
    # Control logic to adjust the throttle
    if speed_error > 0:
        # If the current speed is less than the target, apply throttle
        control.throttle = min(1.0, 0.5 + speed_error * 0.5)  # Adjust factor as needed
    elif speed_error < 0:
        # If the current speed is greater than the target, apply brake
        control.throttle = 0.0
        control.brake = min(1.0, -speed_error * 0.5)  # Adjust braking factor as needed
    else:
        # If we're at the target speed, maintain the throttle
        control.throttle = 0.0
        control.brake = 0.0
    
    # Apply the control
    vehicle.apply_control(control)
    
    # Print the current speed in km/h
    print("Current speed (km/h):", current_speed_mps * 3.6)
    
    # Wait for the next control cycle
    time.sleep(0.1)

control = carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0, reverse=False)
vehicle.apply_control(control)
print(vehicle.get_location())
time.sleep(4.1)
camera_retro.destroy()
camera_rigth.destroy()
vehicle.destroy()


Current speed (km/h): 0.0
Current speed (km/h): 0.009150770120322704
Current speed (km/h): 0.005237858835607768
Current speed (km/h): 0.0015211646561510861
Current speed (km/h): 0.00012958745792275296
Current speed (km/h): 0.001701594761107117
Current speed (km/h): 2.5298078298568725
Current speed (km/h): 5.666091871261597
Current speed (km/h): 7.547644329071045
Current speed (km/h): 8.834461784362793
Current speed (km/h): 9.699872016906738
Current speed (km/h): 10.182388973236085
Current speed (km/h): 10.002523040771484
Current speed (km/h): 9.10631160736084
Current speed (km/h): 8.340555095672608
Current speed (km/h): 8.485519695281983
Current speed (km/h): 8.997414779663087
Current speed (km/h): 9.52106695175171
Current speed (km/h): 9.91614818572998
Current speed (km/h): 10.146154689788819
Current speed (km/h): 9.814140129089356
Current speed (km/h): 9.136282825469971
Current speed (km/h): 9.096529483795166
Current speed (km/h): 9.364295482635498
Current speed (km/h): 9.71022062301

True